# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fdbe0d21d00>
├── 'a' --> tensor([[ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960]])
└── 'x' --> <FastTreeValue 0x7fdbe0d1aaf0>
    └── 'c' --> tensor([[ 0.9232, -0.2992,  0.9040,  0.4002],
                        [ 0.0252,  0.1823, -2.5212, -0.8786],
                        [-1.7766, -1.4758,  1.3622, -0.6583]])

In [4]:
t.a

tensor([[ 0.3944, -0.3479, -0.0087],
        [ 0.7319,  2.1621, -0.3960]])

In [5]:
%timeit t.a

77.4 ns ± 0.171 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fdbe0d21d00>
├── 'a' --> tensor([[ 1.0624, -0.8948,  0.1961],
│                   [ 1.8121, -0.4240, -0.5282]])
└── 'x' --> <FastTreeValue 0x7fdbe0d1aaf0>
    └── 'c' --> tensor([[ 0.9232, -0.2992,  0.9040,  0.4002],
                        [ 0.0252,  0.1823, -2.5212, -0.8786],
                        [-1.7766, -1.4758,  1.3622, -0.6583]])

In [7]:
%timeit t.a = new_value

81.6 ns ± 0.308 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3944, -0.3479, -0.0087],
               [ 0.7319,  2.1621, -0.3960]]),
    x: Batch(
           c: tensor([[ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3944, -0.3479, -0.0087],
        [ 0.7319,  2.1621, -0.3960]])

In [11]:
%timeit b.a

68.6 ns ± 0.0822 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4297,  1.4878, -1.1435],
               [-0.5496,  0.3679,  0.5675]]),
    x: Batch(
           c: tensor([[ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583]]),
       ),
)

In [13]:
%timeit b.a = new_value

581 ns ± 1.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

967 ns ± 2.99 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 34.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

173 µs ± 820 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

169 µs ± 486 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdbe0d8e9d0>
├── 'a' --> tensor([[[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]],
│           
│                   [[ 0.3944, -0.3479, -0.0087],
│                    [ 0.7319,  2.1621, -0.3960]]])
└── 'x' --> <FastTreeValue 0x7fdb29fa1d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 82 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fdb29fa1b50>
├── 'a' --> tensor([[ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960],
│                   [ 0.3944, -0.3479, -0.0087],
│                   [ 0.7319,  2.1621, -0.3960]])
└── 'x' --> <FastTreeValue 0x7fdbb89fa2b0>
    └── 'c' --> tensor([[ 0.9232, -0.2992,  0.9040,  0.4002],
                        [ 0.0252,  0.1823, -2.5212, -0.8786],
                 

In [23]:
%timeit t_cat(trees)

36.1 µs ± 89.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.9 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9232, -0.2992,  0.9040,  0.4002],
                       [ 0.0252,  0.1823, -2.5212, -0.8786],
                       [-1.7766, -1.4758,  1.3622, -0.6583]],
              
                      [[ 0.9232, -0.2992,  0.9040,  0.4002],
                       [ 0.0252,  0.1823, -2.5212, -0.8786],
                       [-1.7766, -1.4758,  1.3622, -0.6583]],
              
                      [[ 0.9232, -0.2992,  0.9040,  0.4002],
                       [ 0.0252,  0.1823, -2.5212, -0.8786],
                       [-1.7766, -1.4758,  1.3622, -0.6583]],
              
                      [[ 0.9232, -0.2992,  0.9040,  0.4002],
                       [ 0.0252,  0.1823, -2.5212, -0.8786],
                       [-1.7766, -1.4758,  1.3622, -0.6583]],
              
                      [[ 0.9232, -0.2992,  0.9040,  0.4002],
                       [ 0.0252,  0.1823, -2.5212, -0.8786],
                       [-1.7766, -1.4758,  1.3622, -0.6583]],

In [26]:
%timeit Batch.stack(batches)

91.7 µs ± 485 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583],
                      [ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583],
                      [ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583],
                      [ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583],
                      [ 0.9232, -0.2992,  0.9040,  0.4002],
                      [ 0.0252,  0.1823, -2.5212, -0.8786],
                      [-1.7766, -1.4758,  1.3622, -0.6583],
                      [ 0.9232, -0.2992,  0.9040,  0.4002],
                   

In [28]:
%timeit Batch.cat(batches)

166 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

393 µs ± 878 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
